In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tqdm.auto import tqdm

tqdm.pandas()

In [3]:

df = pd.read_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_features_2016_2020_v2.parquet.gzip')

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df=df.rename(columns = {'band_x':'band', 'vg_state_x':'vg_state'})

In [6]:
# Split the data set and extract time series features
X = df.loc[:,~df.columns.isin(['veranst_segment','vg_inkasso'])]
y = df['vg_inkasso']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [8]:
display(df.head(5))

,imp_id,gj,import,mufo_referenz_n,barcode_nr,veranst_segment,rekla_jn,vg_datum_von,vg_ort,vg_raum,nutzfall,nutzfall_raum,musikleiter_name,kapelle_name,tarif_nr,tarif_bez,nutzfall_nr,vg_inkasso,inkasso_netto,inkasso_brutto,veranst_geschaeftszeichen,veranst_name,veranst_strasse,veranst_plz,veranst_ort,nutzliznehm_geschaeftszeichen,nutzliznehm_name,nutzliznehm_vorname,nutzliznehm_strasse,nutzliznehm_ort,location,band,promoter,vg_state,vg_raum_clean,vg_raum_wo_stopwords,vg_raum_keywords_x,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,vg_raum_keywords_y,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,vg_state_y,state_bavaria,state_thuringia,state_baden-wuerttemberg,state_rhineland-palatinate,state_brandenburg,state_berlin,state_lower saxony,state_schleswig-holstein,state_hamburg,state_hesse,state_saarland,state_north rhine-westphalia,state_saxony-anhalt,state_saxony,state_mecklenburg-western pomerania,state_bremen,band_y,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9938766,24643,2016,ABRE U INKA 2016,None,5027961235,4,NONRekla,2014-02-08,LISBERG,TURNHALLE SV RW LISBERG,20140208.0,TURNHALLE SV RW LISBERG,STAERK REINHOLD,DAS BOEHMISCHE FEUER,892,U-K (MUSIKER),18293091,210.00,NaN,NaN,1035530900,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V,SCHINDHOLZWEG 17,96194,WALSDORF,1510423300,BAYERISCHER BLASMUSIKVERBAND,None,None,MUENCHEN,LISBERG TURNHALLE SV RW LISBERG,DAS BOEHMISCHE FEUER,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V 96194,BAVARIA,TURNHALLE SV RW LISBERG,turnhalle sv rw lisberg,"[turnhalle, sv, rw, lisberg]",2014,2,5,1,"[turnhalle, sv, rw, lisberg]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,BAVARIA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,DAS BOEHMISCHE FEUER,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939272,24643,2016,ABRE U INKA 2016,None,5053180236,3,NONRekla,2014-02-22,WIPPERDORF,STAATLICHE GRUNDSCHULE WIPPERDORF,20140222.0,STAATLICHE GRUNDSCHULE WIPPERDORF,PATSCHKE RAINER,KYFFHAEUSER TANZ BAN,1087,U-V II. 1 (+II 2),17923031,85.06,NaN,NaN,1374109400,WIPPERDORFER CARNEVAL CLUB E.V,STRASSE DER EINHEIT 6,99752,WIPPERDORF,1374109400,WIPPERDORFER CARNEVAL CLUB,None,STRASSE DER EINHEIT 6,WIPPERDORF,WIPPERDORF STAATLICHE GRUNDSCHULE WIPPERDORF,KYFFHAEUSER TANZ BAN,WIPPERDORFER CARNEVAL CLUB E.V 99752,THURINGIA,STAATLICHE GRUNDSCHULE WIPPERDORF,staatliche grundschule wipperdorf,"[grundschule, wipperdorf]",2014,2,5,1,"[grundschule, wipperdorf]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,THURINGIA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,KYFFHAEUSER TANZ BAN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939279,24643,2016,ABRE U INKA 2016,None,5053345001,3,NONRekla,2014-02-18,DINGOLFING,BMW-WERK DINGOLFING,20140218.0,BMW-WERK DINGOLFING,JUNDMAYER,BERUFSFACHSCHULE,1087,U-V II. 1 (+II 2),17757433,90.20,NaN,NaN,2000026695,BAYERISCHE MOTORENWERKE AG,MENGKOFENER STR. 13,84130,DINGOLFING,2000026695,BAYERISCHE MOTORE

In [9]:
print(len(df.band.unique()))#317011 - unique bands
print(len(df.promoter.unique()))#176456 - unique promoter/organizer
print(len(df.vg_state.unique()))#16 - unique bundesland/states

317011
176456
16


In [33]:
def extract_features(features, target):
    data_path_inkasso_stats_for_features =Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip')

    data_path_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
    data_path_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
    data_path_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

    # stat_var = ['location', 'band_x', 'promoter']
    # stat_var = ['band']
    stat_var = ['vg_state']
    # stat_var = ['promoter']
    # stat_var = ['band']
    df_stat = {}


    # 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
    # 2. Timestamp as index, and have location/band as columns in df
    # 3. Predefine the single df and iterate over it

    def get_descr_stat(row):
        # row represents each line(row) in the dataframe
        def descr_stat(row, var):
            # Initialize a empty pandas dataframe
            result = pd.Series(dtype='float64')

            if not pd.isnull(row[var]):
                #get vg_inkasso collected for past events(that occured before current date of the row) for each location/state, each band and each promoter
                matching_indexes = features[(features['vg_datum_von']<row['vg_datum_von'])&(features[var] == row[var])].index.values.tolist()

                inkasso =  list(target[target.index.isin(matching_indexes)].values)

                # Returns the mean, count, std, max, min and 0.05, 0.10..0.95 percentiles of the list of vg_inkasso values
                # for a specific attribute like location, band, promotor
                result = pd.Series(inkasso, dtype='float64').describe(percentiles=percentiles)
                result = result.add_prefix('{}_'.format(var))

            result.name = row.name
            return result

        descr_stat_result = pd.Series(dtype='float64')
        for v in stat_var:
            descr_stat_result = pd.concat([descr_stat_result, descr_stat(row, v)])

        return descr_stat_result

 
    if data_path_inkasso_stats_for_features.exists() :
        df_featurize_descr_stat = pd.read_parquet(data_path_inkasso_stats_for_features)

    elif data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
        df_state = pd.read_parquet(data_path_vg_state)
        df_band = pd.read_parquet(data_path_band)
        df_promoter = pd.read_parquet(data_path_promoter)

        df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                    how='outer',
                                                                                    on='ID')
    else:
        percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]

        # Each row of the dataframe 'df' is passed as row argument for get_descr_stat function
        df_featurize_descr_stat = features.progress_apply(get_descr_stat, axis=1)
        # df_featurize_descr_stat.to_pickle('./data/export_descr_stat_2016_2020.pkl.bz2', protocol=4)

    # display(df_featurize_descr_stat.head(5))
    return df_featurize_descr_stat

# Splitting 1393792 entries to 128 splits, where each split contains 10889 entries
X_train_splits = np.array_split(X_train,128)
y_train_splits = np.array_split(y_train,128)

df_featurize_descr_stat_result = pd.Series(dtype='float64')
for i in range(0,len(X_train_splits)):
    X_train_slice = X_train_splits[i]
    y_train_slice = y_train_splits[i]
    result = extract_features(X_train_slice, y_train_slice)
    df_featurize_descr_stat_result = pd.concat([df_featurize_descr_stat_result, result])

df_featurize_descr_stat_result.index.name = 'ID'
df_featurize_descr_stat_result = df_featurize_descr_stat_result.drop(0, axis=1) # remove 0th index column with NaN values 
display(df_featurize_descr_stat_result.head(5))

# df_featurize_descr_stat = extract_features(X_train, y_train)

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

...

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

  0%|          | 0/10889 [00:00<?, ?it/s]

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max
ID,,,,,,,,,,,,,,,,,,,,,,,,
12151712,228.0,405.549649,1432.129394,11.02,17.440,18.24,21.800,22.80,35.9800,36.480,45.1000,54.720,67.650,72.50,85.9655,100.460,108.1635,135.242,157.0800,194.956,312.5975,444.854,1388.9360,12403.20
12885094,1629.0,1515.011056,21528.645329,1.10,16.110,18.44,18.640,22.80,23.3000,32.894,37.2800,45.680,54.720,67.44,76.6840,93.200,115.7600,138.944,184.4000,240.268,329.4100,539.366,1250.1480,689747.70
10985840,428.0,1030.554252,9541.693508,12.24,18.440,18.64,20.566,23.05,25.3025,36.520,43.0475,51.344,61.170,69.73,89.3980,109.920,147.5780,189.372,235.8075,303.324,398.8800,691.273,2687.6785,183608.57
10114364,256.0,667.039062,3472.382538,15.50,18.040,18.24,21.800,22.80,30.3300,36.480,45.6000,52.390,61.285,77.52,90.4250,114.000,149.2100,190.990,243.2375,319.200,400.9200,650.375,1445.3350,46779.55
13926216,2055.0,1192.751859,20982.932598,11.02,16.787,18.64,19.710,23.30,31.6450,37.504,45.6000,55.320,63.156,69.90,84.4330,97.386,122.0190,149.824,192.7000,244.650,307.6680,460.460,928.0400,565427.13


In [43]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [45]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [47]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [32]:
df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [49]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
# Without using progress apply and using For-loop to extract features 

data_path_inkasso_stats_for_features =Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip')

data_path_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

# stat_var = ['location', 'band_x', 'promoter']
# stat_var = ['band']
stat_var = ['vg_state']
# stat_var = ['promoter']
# stat_var = ['band']
df_stat = {}



# 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
# 2. Timestamp as index, and have location/band as columns in df
# 3. Predefine the single df and iterate over it

feature_index = 0


if data_path_inkasso_stats_for_features.exists() :
    df_featurize_descr_stat = pd.read_parquet(data_path_inkasso_stats_for_features)

elif data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
    df_state = pd.read_parquet(data_path_vg_state)
    df_band = pd.read_parquet(data_path_band)
    df_promoter = pd.read_parquet(data_path_promoter)

    df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                how='outer',
                                                                                on='ID')
else:

    percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]


    for each_feature in stat_var:
        feature_index = stat_var.index(each_feature)

        feature = stat_var[feature_index]

        # Using DataFrame.copy() create new DataFrame.
        each_feature_df = df[[feature]].copy() #get specific feature in a new dataframe

        # Initialize a empty pandas dataframe of type float64
        result_df = pd.Series(dtype='float64') 

        # Add empty(np.nan) columns to new 'each_feature_df' dataframe
        col = 'count', 'mean', 'std', 'min','5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%', '45%', 
        '50%', '55%', '60%', '65%','70%','75%','80%', '85%', '90%','95%', 'max'
        for each in col:
            cname = each_feature+'_'+each
            each_feature_df[cname] = np.nan
        
        # iterate over each rows in df
        for index, row in tqdm(df.iterrows()):
            
            match_condition = df[each_feature] == row[each_feature]
            filtered_df = df[match_condition] # Filter the dataframe with respect to feature 'each_feature' in iteration

            current_date = row.vg_datum_von
            prev_date_condition = filtered_df.vg_datum_von<current_date
            past_events_df = filtered_df[prev_date_condition] # get events that occured before current row event date
            inkasso = past_events_df['vg_inkasso'].values.tolist() # get all the past inkasso values for specific feature 'each_feature'

            result_df = pd.Series(inkasso).describe(percentiles=percentiles)

            for each in col:     
                cname = each_feature+'_'+each
                each_feature_df.at[index, cname] = result_df[each]
        
            
        df_featurize_descr_stat = each_feature_df

display(df_featurize_descr_stat.head(5))

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
# When the time varied vg_inkasso distribution are extracted for vg_state, band, promoter, merge these features
if data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
    df_state = pd.read_parquet(data_path_vg_state)
    df_band = pd.read_parquet(data_path_band)
    df_promoter = pd.read_parquet(data_path_promoter)

    df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                how='outer',
                                                                                on='ID')
if not data_path_inkasso_stats_for_features.exists() :
    df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip',compression='gzip')